# Baseline RAG Evaluation — Length-Based Chunking

This notebook implements and evaluates a **Baseline RAG system** using traditional length-based chunking.
The document is split into fixed-size chunks (800 characters with 150 overlap), without considering semantic boundaries.

This baseline serves as the foundation for all subsequent experiments. All components (retriever, LLM, evaluation logic) are kept consistent across experiments—only the document preprocessing and retrieval strategies change.

In [1]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Basic_RAG_Semantic_Chunking"

# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

Environment Variables Set:
LANGSMITH_TRACING: true
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: Baseline_RAG


In [3]:
import langchain
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langsmith import traceable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [4]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


## LLM and Embedding Model

In [8]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=4000,
)

In [9]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

## Loading the PDF

In [5]:
@traceable(metadata={"source": "PDF", "file": "NVIDIA_Report_2024.pdf"})
def load_pdf(file_path: str):
    loader = PyPDFLoader(file_path)
    return loader.load()

path ='NVIDIA_Report_2024.pdf'
document = load_pdf(path)

In [60]:
text = document[0].page_content  

# Split into lines and print the first 5
# Just checking whether the correct file was loaded 

print("\n".join(text.split("\n")[:10]))

Table of Contents
UNITED STATESSECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
____________________________________________________________________________________________
FORM 10-K
☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
    For the fiscal year ended January 28, 2024
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
Commission file number: 0-23985


## Chunking the text

In [50]:
def split_text(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        length_function=len,
    )
    docs = text_splitter.split_documents(document)

    # Create and print a table of text chunks
    df = pd.DataFrame({
        "Chunk Index": range(len(docs)),
        "Text Chunk": [doc.page_content for doc in docs]
    })
    
    print("\n===== Top 3 Text Chunks =====\n")
    print(df.head(3).to_string(index=False))  
    print("\n=============================\n")

    
    return docs

In [51]:
docs = split_text(document)
len(docs)


===== Top 3 Text Chunks =====

 Chunk Index                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Text Chunk
           0 Table of Contents\nUNITED STATESSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________________________________

598

## Creating FAISS persistent vector store

In [ ]:
@traceable(metadata={"embedding_model": "text-embedding-3-large", "vector_store": "FAISS"})

def create_vector_store(docs):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local("nvidia_large_baseline") 
    # Saving it locally, so, it can be reloaded later on 
    return vectorstore.as_retriever()

create_vector_store(docs)

## Reloading saved FAISS

In [10]:
# Load the index back

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = FAISS.load_local(
    "nvidia_large_baseline", 
    embeddings,  # Using the same embeddings used during creation
    allow_dangerous_deserialization=True  # Important for loading
)

# Creating a retriver instance out of the loaded FAISS vector store 
retriever = vector_store.as_retriever()

## Basic RAG pipeline 

In [11]:
from langsmith import traceable

@traceable(
    run_type="llm",
    metadata={"ls_provider": "OpenAI", "ls_model_name": "gpt-4o-mini"}
)
def setup_rag_pipeline():
    template = """"
    You are a helpful assistant that answers questions based on the provided context.
    Use the provided context to answer the question.
    Question: {input}
    Context: {context}
    Answer:
    """
    
    # Using this PromptTemplate to answer the query whenever the "context is retrieved"
    prompt = ChatPromptTemplate.from_template(template)
    
    # Setup RAG pipeline
    return (
        {"context": retriever, "input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# Initialize the RAG chain with tracing
rag_chain = setup_rag_pipeline()


Failed to use model_dump to serialize <class 'langchain_core.runnables.base.RunnableSequence'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_community.vectorstores.faiss.FAISS'>)


In [12]:
import pandas as pd
from IPython.display import display

# Define query_rag function with retrieval and display logic
@traceable(metadata={"ls_provider": "OpenAI", "ls_model_name": "gpt-4o-mini"})
def query_rag(question: str, top_k=4):
    """
    Query the RAG pipeline, retrieve relevant documents, and display their similarity scores in descending order.
    Returns both the generated answer and retrieved contexts.
    """
    # Retrieving the top-k chunks with similarity scores
    retrieved_docs_with_scores = retriever.vectorstore.similarity_search_with_score(question, k=top_k)

    # Extracting document details into a structured DataFrame
    df = pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Similarity Score": round(score, 4),  # Rounding for readability
            "Retrieved Chunk": doc.page_content[:1000] + "..."  # Show up to 1000 characters for clarity
        }
        for doc, score in retrieved_docs_with_scores
    ])

    # Ensuring full content is displayed in the DataFrame
    pd.set_option("display.max_colwidth", None)  
    pd.set_option("display.width", 1000)  

    # Displaying the DataFrame in Jupyter
    display(df)

    # Extracting contexts (raw document texts)
    retrieved_contexts = [doc.page_content for doc, _ in retrieved_docs_with_scores]

    # Running the RAG pipeline with the retrieved context
    answer = rag_chain.invoke(question)

    # Returning both the answer and the retrieved contexts
    return {
        "answer": answer,
        "contexts": retrieved_contexts
    }

In [22]:
result = rag_chain.invoke("Who is the CEO of the firm?")

In [23]:
result

'The CEO of the firm is Jen-Hsun Huang.'

### Example usage

In [13]:
response = query_rag("Who is the CEO of the firm?")
print(response["answer"])  # Print the answer
print("\nContexts:", response["contexts"])  # Print the retrieved contexts

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,1.1503,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,11,1.2005,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
2,NVIDIA_Report_2024.pdf,30,1.2557,response team and helping direct the company’s response to and assessment of certain cybersecurity incidents.\nWe have designated a Chief Security Officer that reports to our Senior Vice President of Software Engineering to manage our assessment and management ofmaterial risks from cybersecurity threats. Our Chief Security Officer’s cybersecurity expertise includes over 17 years of combined government and private sector\nassignments.\n31...
3,NVIDIA_Report_2024.pdf,30,1.2701,"matters from our Chief Security Officer and members of our security team.\nOur security efforts are managed by a team of executive cybersecurity, IT, engineering, operations, and legal professionals. We have established a cross-functional leadership team, consisting of executive-level leaders, that meets regularly to review cybersecurity matters and evaluate emerging threats. With\noversight and guidance provided by the cross-functional leadership team, our information security teams refine our practices to address emerging security risksand changes in regulations. Our executive-level leadership team also participates in cybersecurity incident response efforts by engaging with the incident..."


The CEO of the firm is Jen-Hsun Huang.

Contexts: ["Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University.", 'supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter 

## RAG Evaluation QA Dataset from Langsmith

In [35]:
# Importing the LangSmith client
from langsmith import Client

# Initializing the LangSmith client
ls_client = Client()

# Reading the exisiting "RAG_Eval_QA" dataset
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples and print out all questions and reference answers
for example in examples:
    print(example.inputs)  # Print input, i.e. questions
    print(example.outputs)  # Print output, i.e. reference answers


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

### Slowing down request rate

OpenAI's threshold is 200,000 TPM (Token-Per-Minute) for gpt-4o-mini and 30,000 TPM for gpt-4o, meaning that as we process queries in batches, it might be the case that we exceed the threshold, therefore, the rate at which requests are made to OpenAI is artificially slowed down. 


In [38]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_97804/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


## RAG Evaluators

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [20]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag(example["question"])
    return {"answer": response["answer"]}

In [21]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [2]:
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [3]:
grade_prompt_answer_accuracy

StructuredPrompt(input_variables=['correct_answer', 'question', 'student_answer'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'answer_vs_reference', 'lc_hub_commit_hash': '36d7f264ec807f93257fc870a1a40e4f3a1b656c06fba5b8d7eaeddcd6c44cde'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER.\n\nHere is the grade criteria to follow:\n\n(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer.\n(2) Ensure that the student answer does not contain any conflicting statements.\n(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the ground truth answer.\n\nScore:\nA score of 1 means that the student's answer meets all of the criteria. This is the highest (best

In [41]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    llm = model
 
    # Structured prompt
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [25]:
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [40]:
def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [28]:
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [39]:
def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

## Complete Evaluation 

In [42]:
dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="Baseline_RAG",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Baseline_RAG-c6302670' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=b48929b9-51d7-4087-a6cb-bc8d3615174e




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,0.6494,"The NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific\ncomputing, delivering significantly better performance and power efficiency relative to conventional CPU-only approaches. It is deployed in cloud, hyperscale,on-premises and edge data centers. The platform consists of compute and networking offerings typically delivered to customers as systems, subsystems, ormodules, along with software and services...."
1,NVIDIA_Report_2024.pdf,5,0.6353,"Table of Contents\nAt the foundation of the NVIDIA accelerated computing platform are our GPUs, which excel at parallel workloads such as the training and inferencing of neural\nnetworks. They are available in the NVIDIA accelerated computing platform and in industry standard servers from every major cloud provider and server maker.Beyond GPUs, our data center platform expanded to include DPUs in fiscal year 2022 and CPUs in fiscal year 2024. We can optimize across the entire\ncomputing, networking and storage stack to deliver data center-scale computing solutions...."
2,NVIDIA_Report_2024.pdf,6,0.6208,"compute solutions include all three major processing units in AI servers – GPUs, CPUs, and DPUs. GPUs are uniquely suited to AI, and we will continue to addAI-specific features to our GPU architecture to further extend our leadership position. In addition, we offer DGX Cloud, an AI-training-as-a-service platform, andNeMo – a complete solution for building enterprise-ready Large Language Models, or LLMs, using open source and proprietary LLMs created by NVIDIA and\nthird parties. Our AI technology leadership is reinforced by our large and expanding ecosystem in a virtuous cycle. Our computing platforms are available fromvirtually every major server maker and CSP, as well as on our own AI supercomputers. There are over 4.7 million developers worldwide using CUDA and our..."
3,NVIDIA_Report_2024.pdf,6,0.5843,"this platform approach in each of our target markets.\nExtending our technology and platform leadership in AI. We provide a complete, end-to-end accelerated computing platform for AI, addressing both trainingand inferencing. This includes full-stack data center-scale compute and networking solutions across processing units, interconnects, systems, and software. Our..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,12,0.8860,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
1,NVIDIA_Report_2024.pdf,59,0.8422,"In February 2022, NVIDIA and SoftBank Group Corp, or SoftBank, announced the termination of the Share Purchase Agreement whereby NVIDIA would have\nacquired Arm from SoftBank. The parties agreed to terminate it due to significant regulatory challenges preventing the completion of the transaction. Werecorded an acquisition termination cost of $1.4 billion in fiscal year 2023 reflecting the write-off of the prepayment provided at signing.\nNote 3 - Leases\nOur lease obligations primarily consist of operating leases for our headquarters complex, domestic and international office facilities, and data center space, withlease periods expiring between fiscal years 2025 and 2035.\nFuture minimum lease payments under our non-cancelable operating leases as of January 28, 2024, are as follows:..."
2,NVIDIA_Report_2024.pdf,11,0.8149,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.8052,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,21,0.7992,"Table of Contents\nWe are monitoring the impact of the geopolitical conflict in and around Israel on our operations, including the health and safety of our approximately 3,700employees in the region who primarily support the research and development, operations, and sales and marketing of our networking products. Our operatingexpenses in fiscal year 2024 include expenses for financial support to impacted employees and charitable activity. We believe our global supply chain for our..."
1,NVIDIA_Report_2024.pdf,34,0.7741,"manage product availability and costs with our vendors. \nIsrael and Hamas Conflict\nWe are monitoring the impact of the geopolitical conflict in and around Israel on our operations, including the health and safety of our approximately 3,700\nemployees in the region who primarily support the research and development, operations, and sales and marketing of our networking products. Our operatingexpenses in fiscal year 2024 include expenses for financial support to impacted employees and charitable activity. We believe our global supply chain for our..."
2,NVIDIA_Report_2024.pdf,21,0.7664,"networking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or others\nmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense toour business; however, if the conflict is further extended, it could impact future product development, operations, and revenue or create other uncertainty for ourbusiness...."
3,NVIDIA_Report_2024.pdf,34,0.7512,"networking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or othersmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense to\nour business; however, if the conflict is further extended, it could impact future product development, operations, and revenue or create other uncertainty for ourbusiness.\n35..."


...

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,6,0.7180,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
1,NVIDIA_Report_2024.pdf,4,0.6938,"Professional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
2,NVIDIA_Report_2024.pdf,3,0.6685,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
3,NVIDIA_Report_2024.pdf,4,0.5249,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,10,0.9695,"identity of the reporter and provide a mechanism for reporters to follow an investigation and receive responses.\nWe want NVIDIA to be a place where people can build their careers over their lifetime. Our employees tend to come and stay. In fiscal year 2024, our overall\nturnover rate was 2.7%.\nCompensation, Benefits, and Well-Being\nOur compensation program rewards performance and is structured to encourage employees to invest in the Company’s future. Employees receive equity, except\nwhere unavailable due to local regulations, that is tied to the value of our stock price and vests over time to retain employees while simultaneously aligning theirinterests with those of our shareholders...."
1,NVIDIA_Report_2024.pdf,11,0.9301,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
2,NVIDIA_Report_2024.pdf,11,0.7381,"• Track equity and parity in retention, promotions, pay, and employee engagement scores; and\n• Measuring year over year progress and providing leadership visibility on diversity efforts.\nAs of the end of fiscal year 2024, our global workforce was 79% male, 20% female, and 1% not declared, with 6% of our workforce in the United States\ncomposed of Black or African American and Hispanic or Latino employees.\nFlexible Working Environment\nWe support a flexible work environment, understanding that many employees want the ability to work from home under certain conditions. This flexibility\nsupports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work...."
3,NVIDIA_Report_2024.pdf,10,0.5330,"scientists, engineers, and technical and non-technical staff.\nRecruitment\nAs the demand for global technical talent continues to be competitive, we have grown our technical workforce and have been successful in attracting top talentto NVIDIA. We have attracted talent globally through our strong employer brand and differentiated hiring strategies for college, professional, and leadershiptalent. Our workforce is 83% technical and 49% hold advanced degrees. Additionally, we have increased focus on diversity recruiting, resulting in an increase in\nglobal female hiring in each channel. Our own employees help to surface top talent, with over 40% of our new hires in fiscal year 2024 coming from employeereferrals.\nDevelopment and Retention..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,7,0.7647,"Table of Contents\ntier-1 suppliers, and start-ups. Our AV solution also includes the GPU-based hardware required to train the neural networks before their in-vehicle deployment,as well as to re-simulate their operation prior to any over-the-air software updates. We believe our comprehensive, top-to-bottom and end-to-end approach will\nenable the transportation industry to solve the complex problems arising from the shift to autonomous driving.\nLeveraging our intellectual property, or IP. We believe our IP is a valuable asset that can be accessed by our customers and partners through license anddevelopment agreements when they desire to build such capabilities directly into their own products or have us do so through a custom development. Such..."
1,NVIDIA_Report_2024.pdf,6,0.7257,"Table of Contents\nWe are working with several hundred partners in the automotive ecosystem including automakers, truck makers, tier-one suppliers, sensor manufacturers,\nautomotive research institutions, HD mapping companies, and startups to develop and deploy AI systems for self-driving vehicles. Our unified AI computingarchitecture starts with training deep neural networks using our Data Center computing solutions, and then running a full perception, fusion, planning, and controlstack within the vehicle on the NVIDIA DRIVE Hyperion platform. DRIVE Hyperion consists of the high-performance, energy efficient DRIVE AGX computing..."
2,NVIDIA_Report_2024.pdf,6,0.7131,"the transportation industry. The algorithms required for autonomous driving - such as perception, localization, and planning - are too complex for legacy hand-coded approaches and will use multiple neural networks instead. In addition, EV makers are looking for next-generation centralized car computers that integratea wide range of intelligent functions into a single AI compute platform. Therefore, we provide an AI-based hardware and software solution, designed and\nimplemented from the ground up based on automotive safety standards, for the AV and EV market under the DRIVE brand, which we are bringing to marketthrough our partnerships with automotive OEMs,\n7..."
3,NVIDIA_Report_2024.pdf,6,0.6599,"hardware, a reference sensor set that supports full self-driving capability as well as an open, modular DRIVE software platform for autonomous driving, mapping,and parking services, and intelligent in-vehicle experiences.\nIn addition, we offer a scalable data center-based simulation solution, NVIDIA DRIVE Sim, based on NVIDIA Omniverse software, for digital cockpit\ndevelopment, as well as for testing and validating a self-driving platform. Our unique end-to-end, software-defined approach is designed for continuousinnovation and continuous development, enabling cars to receive over-the-air updates to add new features and capabilities throughout the life of a vehicle.\nBusiness Strategies\nNVIDIA’s key strategies that shape our overall business approach include:..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,0.7459,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
1,NVIDIA_Report_2024.pdf,49,0.6642,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
2,NVIDIA_Report_2024.pdf,35,0.6613,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
3,NVIDIA_Report_2024.pdf,35,0.5506,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,12,0.9087,"Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinnesota, an M.S.E.E. degree from the California Institute of Technology and an M.B.A. degree from Harvard Business School.\nDebora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement..."
1,NVIDIA_Report_2024.pdf,12,0.8722,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
2,NVIDIA_Report_2024.pdf,11,0.6880,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.5491,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
